# Load Tests on FAST API

In [1]:
import time
import requests
from concurrent.futures import ThreadPoolExecutor

# ────────────────────────────────
# Configuration
# ────────────────────────────────
FASTAPI_URL = "http://129.114.25.110:8000/generate"
PROMPT_TEXT = "Summarize clause 7.2 of this document:"
DEFAULT_PAYLOAD = {"prompt": PROMPT_TEXT, "max_new_tokens": 50}

# pattern: number of simultaneous clients in each stage
load_pattern = [1, 2, 3, 5, 3, 2, 1]
# how long (seconds) each stage should run
delay_between_steps = 30

# ────────────────────────────────
# Worker: keep firing requests for duration_sec
# ────────────────────────────────
def send_continuous_requests(duration_sec, payload=DEFAULT_PAYLOAD):
    t0 = time.time()
    while time.time() - t0 < duration_sec:
        try:
            resp = requests.post(FASTAPI_URL, json=payload, timeout=10)
            # you can log status or response time here if you like
            # e.g. print(resp.status_code, resp.elapsed.total_seconds())
        except Exception as e:
            print("Request error:", e)

# ────────────────────────────────
# Stage runner: spins up N concurrent senders
# ────────────────────────────────
def run_load_stage(concurrent_workers, duration_sec):
    print(f"➤ Running stage: {concurrent_workers} workers for {duration_sec}s")
    with ThreadPoolExecutor(max_workers=concurrent_workers) as executor:
        futures = [
            executor.submit(send_continuous_requests, duration_sec)
            for _ in range(concurrent_workers)
        ]
        # wait for them all to finish
        for f in futures:
            f.result()

# ────────────────────────────────
# Execute the ramp‐up/down pattern
# ────────────────────────────────
if __name__ == "__main__":
    for workers in load_pattern:
        run_load_stage(workers, delay_between_steps)
    print("Load test complete")


➤ Running stage: 1 workers for 30s
Request error: HTTPConnectionPool(host='129.114.25.110', port=8000): Max retries exceeded with url: /generate (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7fb50831bc40>, 'Connection to 129.114.25.110 timed out. (connect timeout=10)'))
Request error: HTTPConnectionPool(host='129.114.25.110', port=8000): Max retries exceeded with url: /generate (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7fb50836c430>, 'Connection to 129.114.25.110 timed out. (connect timeout=10)'))
Request error: HTTPConnectionPool(host='129.114.25.110', port=8000): Max retries exceeded with url: /generate (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7fb50836cb50>, 'Connection to 129.114.25.110 timed out. (connect timeout=10)'))
➤ Running stage: 2 workers for 30s
Request error: HTTPConnectionPool(host='129.114.25.110', port=8000): Max retries exceeded with url: /generate (Caused by Conne

KeyboardInterrupt: 